In [29]:

let schematic2D = 
    File.ReadLines "./input"
    |> Seq.map (Seq.toArray)
    |> Seq.toArray
    |> array2D 

//Map of touching numbers by each gear coordinates
let mutable gears: Map<(int * int), List<int>> = Map.empty

let toBoundingBox (c,i,j) =
    let mutable endIndex = j

    while (endIndex < (Array2D.length2(schematic2D)-1) && Char.IsDigit schematic2D.[i,endIndex+1]) do
        endIndex <- endIndex + 1

    let number = schematic2D[i,j..endIndex]

    let topLeft = ((if i = 0 then 0 else i-1),(if j = 0 then 0 else j-1))
    let bottomRight = ((if i = Array2D.length1(schematic2D)-1 then Array2D.length1(schematic2D)-1 else (i+1)), (if endIndex = Array2D.length2(schematic2D)-1 then Array2D.length2(schematic2D)-1 else (endIndex+1)))

    (number |> String |> int, topLeft, bottomRight)

let isValidPart (n: int, (x1,y1), (x2,y2)) =
    let mutable valid = false
    let slice = schematic2D[x1..x2,y1..y2]

    for r = x1 to x2 do
        for c = y1 to y2 do
            if not(Char.IsDigit schematic2D[r,c] || schematic2D[r,c] = '.') then valid <- true 
            // handle gears in global mutable var as side effect when looking for symbols, VERY NAUGHTY
            if schematic2D[r,c] = '*' then gears <- gears.Add((r,c), match gears.TryFind (r,c) with
                                                                        | Some(x) -> List.append x [n]
                                                                        | None -> [n])

    valid


File.ReadLines "./input"
|> Seq.mapi ((fun rowIndex row -> row.ToCharArray() |> Seq.mapi (fun columnIndex char -> (char, rowIndex, columnIndex))))
|> Seq.collect (fun x -> x)
// Keep only the first character of each number
|> Seq.filter (fun (c,i,j) -> Char.IsDigit c && ((j = 0) || not (Char.IsDigit schematic2D.[i,j-1])))
// calculate the a square arround the number
|> Seq.map toBoundingBox
|> Seq.filter isValidPart
|> Seq.sumBy (fun (n, (x1,y1), (x2,y2)) -> n)

540212

In [30]:
gears
|> Map.filter (fun k v -> List.length(v) = 2) // has two neighbors
|> Map.values
|> Seq.map (fun list -> list |> Seq.reduce (fun a b -> a*b)) // multiply the neighbors
|> Seq.sum

87605697